In [ ]:
# Install Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Download Spark 3.3.2 (working mirror)
!wget -q https://archive.apache.org/dist/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz

# Extract Spark
!tar -xvzf spark-3.3.2-bin-hadoop3.tgz

# Install findspark
!pip install -q findspark

In [ ]:
import os
import findspark

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.2-bin-hadoop3"

findspark.init()

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("US Gov Spending Analysis") \
    .getOrCreate()

spark

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
df = spark.read.csv("gov_spending.csv", header=True, inferSchema=True)
df.show()

In [ ]:
df = spark.read.csv("gov_spending.csv", header=True, inferSchema=True)
df.show()

In [ ]:
# Drop any rows with missing values (if needed)
df_clean = df.dropna()

# View data types
df_clean.printSchema()

# Optional: Show total number of rows
print("Total Records:", df_clean.count())

In [ ]:
df_clean.groupBy("Make").count().orderBy("count", ascending=False).show()

In [ ]:
df_clean.groupBy("Model Year").count().orderBy("Model Year").show()

In [ ]:
df_clean.groupBy("City").count().orderBy("count", ascending=False).show(10)

In [ ]:
df_clean.groupBy("Electric Vehicle Type").count().orderBy("count", ascending=False).show()

In [ ]:
df_clean.groupBy("County").count().orderBy("count", ascending=False).show()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

top_makes = df_clean.groupBy("Make").count().orderBy("count", ascending=False).limit(10).toPandas()

plt.figure(figsize=(10,6))
plt.bar(top_makes['Make'], top_makes['count'])
plt.title("Top 10 EV Makes in Washington")
plt.xticks(rotation=45)
plt.xlabel("Make")
plt.ylabel("Count")
plt.tight_layout()
plt.show()

In [ ]:
!apt-get install graphviz -qq
!pip install graphviz

In [ ]:
from graphviz import Digraph

dot = Digraph()

dot.attr(rankdir='LR')  # Left to Right layout

# Define nodes
dot.node('A', 'CSV File\n(gov_spending.csv)')
dot.node('B', 'Google Colab')
dot.node('C', 'PySpark DataFrame')
dot.node('D', 'Data Cleaning\n& Aggregation')
dot.node('E', 'Spark Analysis\n(Year, Make, City, Type)')
dot.node('F', 'Visualizations\n(Matplotlib Charts)')
dot.node('G', 'Insights\n& Trends')

# Define edges
dot.edge('A', 'B')
dot.edge('B', 'C')
dot.edge('C', 'D')
dot.edge('D', 'E')
dot.edge('E', 'F')
dot.edge('F', 'G')

# Display the diagram
dot.render('pipeline_diagram', format='png', cleanup=False)
dot

In [ ]:

# ---------------- EV-registration quick-look ----------------
# Upload your gov_spending.csv file when prompted and then run this cell.


# 1️⃣  Imports & load
import pandas as pd
import matplotlib.pyplot as plt

csv_name = next(iter(uploaded))
df = pd.read_csv(csv_name)

# 2️⃣  Registrations by model year
year_counts = df['Model Year'].value_counts().sort_index()
display(year_counts.to_frame('Registrations'))
plt.figure()
plt.plot(year_counts.index, year_counts.values, marker='o')
plt.title('Vehicle registrations by model year')
plt.xlabel('Model year')
plt.ylabel('Number of vehicles')
plt.grid(True)
plt.show()

# 3️⃣  Top 10 makes
top_makes = df['Make'].value_counts().head(10)
display(top_makes.to_frame('Registrations'))
plt.figure()
plt.barh(top_makes.index[::-1], top_makes.values[::-1])
plt.title('Top 10 EV makes')
plt.xlabel('Number of vehicles')
plt.tight_layout()
plt.show()

# 4️⃣  BEV vs PHEV share
type_counts = df['Electric Vehicle Type'].value_counts()
display(type_counts.to_frame('Registrations'))
plt.figure()
plt.pie(type_counts.values,
        labels=type_counts.index,
        autopct='%1.1f%%',
        startangle=140)
plt.title('Battery-electric vs Plug-in hybrid')
plt.show()

# 5️⃣  Median range over time
range_median = (df.groupby('Model Year')['Electric Range']
                  .median()
                  .dropna()
                  .sort_index())
display(range_median.to_frame('Median Range (mi)'))
plt.figure()
plt.plot(range_median.index, range_median.values, marker='o')
plt.title('Median electric range by model year')
plt.xlabel('Model year')
plt.ylabel('Median range (miles)')
plt.grid(True)
plt.show()


1. Forecast total registrations per model year

In [ ]:
from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# --- prep the target series ---
year_counts = df['Model Year'].value_counts().sort_index()
X = year_counts.index.values.reshape(-1, 1)         # 2-D for sklearn
y = year_counts.values

# --- fit simple linear model ---
lin = LinearRegression().fit(X, y)

# --- predict next 3 years ---
future_years = np.array([2025, 2026, 2027]).reshape(-1, 1)
future_pred  = lin.predict(future_years)

# --- combine & plot ---
proj = pd.Series(future_pred, index=future_years.flatten(), name='Predicted')
combined = pd.concat([year_counts.rename('Actual'), proj])
combined.plot(marker='o', figsize=(8,4))
plt.title('EV registrations • history vs 3-yr linear projection')
plt.ylabel('Vehicles'); plt.xlabel('Model year'); plt.grid(True)
plt.show()

print(combined.tail(3))

2. Forecast median electric range

In [ ]:
from sklearn.linear_model import LinearRegression
import numpy as np

range_median = (df.groupby('Model Year')['Electric Range']
                  .median().dropna().sort_index())

X = range_median.index.values.reshape(-1,1)
y = range_median.values

lin = LinearRegression().fit(X, y)
future_pred = lin.predict(future_years)

proj = pd.Series(future_pred, index=future_years.flatten(), name='Predicted')
combined = pd.concat([range_median.rename('Actual'), proj])

combined.plot(marker='o', figsize=(8,4))
plt.title('Median EPA range • history vs 3-yr projection')
plt.ylabel('Miles'); plt.xlabel('Model year'); plt.grid(True)
plt.show()

print(combined.tail(3))

3. Forecast BEV share vs PHEV

In [ ]:
share = (df.groupby('Model Year')['Electric Vehicle Type']
           .value_counts(normalize=True)
           .loc[:, 'Battery Electric Vehicle (BEV)']
           .sort_index())

X = share.index.values.reshape(-1,1)
y = share.values

lin = LinearRegression().fit(X, y)
future_pred = lin.predict(future_years).clip(0,1)   # keep inside 0–1

proj = pd.Series(future_pred, index=future_years.flatten(), name='Predicted BEV share')
combined = pd.concat([share.rename('Actual BEV share'), proj])

combined.plot(marker='o', figsize=(8,4))
plt.title('BEV fleet share • history vs 3-yr projection')
plt.ylabel('Share of registrations'); plt.xlabel('Model year'); plt.grid(True)
plt.show()

print((combined.tail(3)*100).round(1).astype(str) + '%')